In [1]:
from __future__ import print_function

"""
A reference implementation of cavity quantum electrodynamics 
configuration interactions singles.
"""

__authors__   = ["Jon McTague", "Jonathan Foley"]
__credits__   = ["Jon McTague", "Jonathan Foley"]

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-01-15"

# ==> Import Psi4, NumPy, & SciPy <==
import psi4
import numpy as np
import scipy.linalg as la
import time
from helper_cqed_rhf import *

# Set Psi4 & NumPy Memory Options
psi4.set_memory('2 GB')
#psi4.core.set_output_file('output.dat', False)

numpy_memory = 2

# basis set etc
psi4.set_options({'basis':        'def2-tzvppd',
                  'scf_type':     'pk',
                  'reference':    'rhf',
                  'mp2_type':     'conv',
                  'save_jk': True,
                  'e_convergence': 1e-10,
                  'd_convergence': 1e-10})

NaF_string = """

0 1
    NA           0.000000000000     0.000000000000    -0.875819904077
    F            0.000000000000     0.000000000000     1.059820520433
no_reorient
#nocom
symmetry c1
"""

NaCl_string = """

0 1
    NA           0.000000000000     0.000000000000    -1.429419641344
    CL           0.000000000000     0.000000000000     0.939751385626
no_reorient
#nocom
symmetry c1
"""

expected_NaF =  -261.371070718358
expected_NaCl = -621.438985539266

# electric field
Ex = 0.
Ey = 0.
Ez = 0.01

lam = np.array([Ex, Ey, Ez])
# run cqed_rhf on NaF and compare to expected answer
cqed_rhf_dict = cqed_rhf(lam, NaF_string)
em_cqed_rhf_e = cqed_rhf_dict['cqed_rhf_energy']
em_rhf_e = cqed_rhf_dict['rhf_energy']
assert np.isclose(em_cqed_rhf_e, expected_NaF,5e-5)
print("def2-tzvppd RHF energy of NaF:               ", em_rhf_e)
print("def2-tzvppd CQED-RHF energy of NaF:          ", em_cqed_rhf_e)
print("reference def2-tzvppd CQED-RHF energy of NaF:", expected_NaF)

cqed_rhf_dict = cqed_rhf(lam, NaCl_string)
em_rhf_e = cqed_rhf_dict['rhf_energy']
em_cqed_rhf_e = cqed_rhf_dict['cqed_rhf_energy']
assert np.isclose(em_cqed_rhf_e, expected_NaCl,5e-5)
print("def2-tzvppd RHF energy of NaCl:               ", em_rhf_e)
print("def2-tzvppd CQED-RHF energy of NaCl:          ", em_cqed_rhf_e)
print("reference def2-tzvppd CQED-RHF energy of NaCl:", expected_NaCl)
# change to the cc-pVDZ basis set
psi4.set_options({'basis':        'cc-pVDZ'})
# template for the z-matrix for MgH+
mol_tmpl = """Mg
H 1 **R**
symmetry c1
1 1"""
# array of bondlengths for MgH+
#r_array = np.array([1.0, 1.1, 1.2, 1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0,2.1,2.2,2.3])
r_array = np.array([1.0, 1.1, 1.2, 1.3,1.4,1.5,1.6])
# lambda arrays
l_big = np.array([0.0, 0.0, 0.1])
l_med = np.array([0.0, 0.0, 0.01])


-261.37134961184677

Start SCF iterations:

Canonical RHF One-electron energy = -423.8943127302466110
CQED-RHF One-electron energy = -423.8915963779146523
Nuclear repulsion energy = 27.0652251280564968
Dipole energy = 0.0004869813724925
SCF Iteration   1: Energy = -261.3711084073713664   dE = -2.61371E+02   dRMS = 2.23432E-06
SCF Iteration   2: Energy = -261.3711084455142100   dE = -3.81428E-08   dRMS = 1.52094E-06
Performed QED-RHF on the following molecule


0 1
    NA           0.000000000000     0.000000000000    -0.875819904077
    F            0.000000000000     0.000000000000     1.059820520433
no_reorient
#nocom
symmetry c1

Total time for SCF iterations: 0.277 seconds 

QED-RHF   energy: -261.37110845 hartree
Psi4  SCF energy: -261.37134961 hartree
def2-tzvppd RHF energy of NaF:                -261.37134961184677
def2-tzvppd CQED-RHF energy of NaF:           -261.3711084455142
reference def2-tzvppd CQED-RHF energy of NaF: -261.371070718358
-621.4409452668015

Start SCF iterati